# Schnorr digital signatures and verification
## Key pair generation

- $E$ is an elliptic curve, $N$ is the order of the curve
- $G$ is a generator and $h$ is an hash function
- The user chooses his secret key $d$, $0 < d < N$
- The user computes the point $P = dG = (P_x, P_y)$: his public key is the $x$-coordinate, $P_x$

In [1]:
import create_keypair as ckp
import schnorr_lib as sl

n_keys = int(input("Insert the number of keys to be generated:"))

ckp.create_keypair(n_keys)["users"]

[{'privateKey': '1bd07d64e130f7367cc38215135d39bdaa729a89de92be39aff07f4f439a2e91',
  'publicKey': 'bceac5d875dec7d0e84db68f201bc1e52df82d7cdcc5e743f2b3b3ed2a1e8124'},
 {'privateKey': 'a33554fe3485f54076d40c32571465f5c1152b66c2de35b57f12ac810cc7635e',
  'publicKey': '63e748d3223df163f36d275990a660a3e57ec2aa4fa0fa906ce4f8de1a50f553'}]

## Schnorr Sign

- $M$ message, $d$ is the private key, the $x$-coordinate of $P = dG$ is the public key
- $A$, the signer, picks a random $k$, $1 < k < n$ and calculates $R = kG$
- A computes $e = h(R || P || M)$
- A computes $s = (k + ed) \mod n$
- The signature is the pair $(R_x, s)$ 

In [2]:
import create_keypair as ckp
import schnorr_lib as sl

user = ckp.create_keypair(1)["users"]

M = input("Insert the message to sign:")
M = sl.sha256(M.encode())

sig = sl.schnorr_sign(M, user[0]["privateKey"])

print("PublicKey =",user[0]["publicKey"])
print("Signature =",sig.hex())

PublicKey = bdd526e049d73befe377eb678a61c221ec15a5cb1a9a4102fd43e7871723c61d
Signature = 77a6f220a154ea16a4574462523560b05671625f67a3a1b582d038cf2aea5097e2114923c85d8e8d5303ad1a47164e8e7ad4ba64944cc881a4b2e5962bad3603


## Schnorr MuSig-1

- $L = h(P_1 || ... ||P_n)$, where n is the number of users and $P_i$ is the public key of the $i^{th}$ user
- Every user $i$ computes the quantity $a_i = h(L||P_i)$
- The aggregate $\tilde{X} = \sum\limits_{i=1}^{n} a_i P_i$ is a public parameter
- Every user chooses $r_i$ and computes $R_i = r_i G$
- Every user computes the point $R = R_1+...+R_n =(R_x,R_y)$
- Every user computes $c = h(R || \tilde{X}|| M)$
- Every user computes $s_i = r_i + cd_i a_i \mod N$, then the aggregate is $s=s_1+...+s_n \mod N$
- The signature is the couple $(R_x,s)$

In [3]:
import create_keypair as ckp
import schnorr_lib as sl

n_keys = int(input("Insert the number of keys to be generated:"))
users = ckp.create_keypair(n_keys)["users"]

M = input("Insert the message to sign:")
M = sl.sha256(M.encode())

sig, X = sl.schnorr_musig_sign(M, users)

print("Aggregated key =",X.hex())
print("Signature =",sig.hex())

Aggregated key = b37399139335be8a3a8dfb77ee2c1ea3e60f5abf429f5244a2923f40aba0cfaf
Signature = 27993dff5e0ce2259ecc2dd3768074cd0af2e9899b0da5de2651c5d1149f6adde12f6c89cefc4c3479b6857fa20f7d866d22e0f2648318ff3606854101682a81


## Schnorr MuSig-2

- Compute $L$, $a_i = h(L||P_i )$ and $\tilde{X}$ as before
- Each user $i$ chooses $\nu$ different nonces $r_{i,1}, . . . , r_{i,ν}$ and computes the points
$R_{i,j} = r_{i,j}G$, $\forall j \in \{1,...,\nu\}$
- Compute $R_j = \sum\limits_{i=1}^{n} R_{i,j}, \; \forall j \in \{1,...,\nu\}$
- Compute the hash $b = h(R_1||...||R_{\nu}|| \tilde{X} || M)$, then compute $R = \sum\limits_{j=1}^{\nu} b^{j-1} R_{j}$
- Compute the hash $c = h(R || \tilde{X}||M)$, then every user $i$ computes
$s_i=c a_id_i+\sum\limits_{j=1}^{\nu} r_{i,j}\; b^{j−1} \mod N$
- Compute $s=s_1+...+s_n \mod N$
- The signature is the couple $(R_x,s)$.

In [ ]:
import create_keypair as ckp
import schnorr_lib as sl

n_keys = int(input("Insert the number of keys to be generated:"))
users = ckp.create_keypair(n_keys)["users"]

M = input("Insert the message to sign:")
M = sl.sha256(M.encode())

sig, X = sl.schnorr_musig2_sign(M, users)

print("Aggregated key =",X.hex())
print("Signature =",sig.hex())

## Verify Schnorr

- $B$, the receiver, computes the point $sG$
- $B$ computes $e = h(R || P || M)$ $(P$ is equal to $\tilde{X}$ in the MuSig cases$)$
- if $sG = R + eP$, the signature is valid  $(P$ is equal to $\tilde{X}$ in the MuSig cases$)$

In [4]:
import schnorr_lib as sl

M = input("Insert the message to verify:")
M = sl.sha256(M.encode())

pubkey = input("Insert the public key (or the aggregated key if MuSig was used):")
pubkey_bytes = bytes.fromhex(pubkey)

sig = input("Insert the generated sign:")
sig_bytes = bytes.fromhex(sig)

result = sl.schnorr_verify(M, pubkey_bytes, sig_bytes)

if result:
    print("The signature is VALID for this message and this public key")
else:
    print("The signature is NOT VALID for this message and this public key")

The signature is VALID for this message and this public key
